# Image Labeling (Edu)

## Imports and Parameters

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

sys.path.append("../Library/")
import image_download as imd
import image_manipulation as ima
import db_connection as dbcon

#%matplotlib inline
%load_ext autoreload
%autoreload 2

#from PIL import Image
#import imageio
#from IPython.display import display, clear_output
#
#from dotenv import load_dotenv
#import os
#load_dotenv("../.env")
#import csv

In [2]:
# parameters
GMAPS_API_KEY = os.getenv('GMAPS_API_KEY')
IMG_FOLDER = os.getenv('MFP_IMG_FOLDER')
DATASET = 'gmaps'

# connect to db
db = dbcon.connect("../credentials/mlab_db.txt","mfp")
images_lib_col = db["images_lib"]

DB Credentials from file
DB connected successfully!!!


## Load filenames from DB

In [53]:
MAX_IMG_TO_LABEL = 5 # limit the images to label

query = {
    "$and" : [
        #{ "name":"Spain" },
        {"label_multi_er": {"$exists": True} },
        {"gist": {"$exists": True} },
        #{"gist": {"$exists": False} }
    ]  
}

images_metadata = [img_metadata for img_metadata in images_lib_col.find(query)]
print(len(images_metadata), "query matches\n")

images_filenames = [image["filename"] for image in images_metadata[:MAX_IMG_TO_LABEL]]
print(len(images_filenames), "images to label:")
for image in images_filenames[:10]:
    print(image)
    
# Get image info of all the files of area name provided above (filenames, images, np.array)
images_info = ima.load_images_from_gdrive(images_filenames, IMG_FOLDER + DATASET + "/")

40 query matches

5 images to label:
US1_45.21160965529817_-93.54554509042913_13_640.png
US1_45.21160965529817_-93.54554509042913_14_640.png
US1_45.21160965529817_-93.54554509042913_15_640.png
US1_45.21160965529817_-93.54554509042913_16_640.png
US1_52.20482183860124_-92.48891258998856_13_640.png


## Start labeling app and save csv

THIS WILL OPEN AN INTERACTIVE WINDOW FOR LABELLING THE IMAGES!!!

FIRST PUT IN YOUR NAME AND THEN GO AHEAD WITH LABELLING

In [54]:
filename_upload_to_db = 'ER - ' + str(datetime.datetime.today())[:19].replace(':','.') + '.csv'

images_info = ima.add_labels_and_save_csv(images_info, IMG_FOLDER, filename_upload_to_db)

Output file: ER - 2019-03-21 23.54.38.csv


## Load labels into DB

In [9]:
dbcon.write_labels_from_csv_to_db(images_lib_col, IMG_FOLDER, filename_upload_to_db)

loading labels label_multi_er and label_binary_er from file ER - 2019-03-21 23.29.29.csv to db ...
5 labels added to db!


---

### Check binary labels

In [111]:
label_name = 'label_binary_er'
query = {label_name: '0'}

doc = images_lib_col.find(query)
for x in doc:
    print(x['filename'], "has label", x[label_name])

US1_52.20482183860124_-92.48891258998856_13_640.png has label 0
US1_52.20482183860124_-92.48891258998856_14_640.png has label 0
US1_52.20482183860124_-92.48891258998856_15_640.png has label 0
US1_52.20482183860124_-92.48891258998856_16_640.png has label 0
US1_33.01060691483008_-118.04101400786547_13_640.png has label 0
US2_41.76932454759587_-105.42877312370616_14_640.png has label 0
US2_41.76932454759587_-105.42877312370616_15_640.png has label 0
US2_41.76932454759587_-105.42877312370616_16_640.png has label 0


In [112]:
label_name = 'label_binary_er'
query = {label_name: '1'}

doc = images_lib_col.find(query)
for x in doc:
    print(x['filename'], "has label", x[label_name])

US1_45.21160965529817_-93.54554509042913_13_640.png has label 1
US1_45.21160965529817_-93.54554509042913_14_640.png has label 1
US1_45.21160965529817_-93.54554509042913_15_640.png has label 1
US1_45.21160965529817_-93.54554509042913_16_640.png has label 1
US1_33.01060691483008_-118.04101400786547_14_640.png has label 1
US1_33.01060691483008_-118.04101400786547_15_640.png has label 1
US1_33.01060691483008_-118.04101400786547_16_640.png has label 1
US1_50.53186589235239_-108.35386180596005_13_640.png has label 1
US1_50.53186589235239_-108.35386180596005_14_640.png has label 1
US1_50.53186589235239_-108.35386180596005_15_640.png has label 1
US1_50.53186589235239_-108.35386180596005_16_640.png has label 1
US1_45.14093597874625_-101.4861640355236_13_640.png has label 1
US1_45.14093597874625_-101.4861640355236_14_640.png has label 1
US1_45.14093597874625_-101.4861640355236_15_640.png has label 1
US1_45.14093597874625_-101.4861640355236_16_640.png has label 1
US2_32.98997192625039_-84.9187594